<a href="https://colab.research.google.com/github/Sizwe100/Python-Random-Forest/blob/main/Copy_of_NDVI_TimeSeries_Image_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api
!earthengine authenticate

E0000 00:00:1741352145.164634     397 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741352145.197389     397 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
W0307 12:55:59.322051 140256059666432 _default.py:711] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20h

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-sizwepmthimkhulu')

In [4]:
from google.colab import drive
drive.mount('/content/drive')
#Insert the directoryimport sys
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [6]:
!pip install folium
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 28.3 MB/s eta 0:00:00


In [7]:
import folium
import rasterio
from rasterio.mask import mask
import geopandas as gpd

In [ ]:
def shapefile_to_ee_geometry(shapefile_folder_path):
    gdf = gpd.read_file(shapefile_folder_path)

    # Validate geometry: Ensure each polygon has at least 3 points
    for index, row in gdf.iterrows():
        if len(row.geometry.exterior.coords) < 3:
            # Handle invalid geometries (e.g., remove or repair)
            print(f"Warning: Polygon at index {index} has less than 3 points. Skipping or fixing this geometry might be necessary.")
            gdf.drop(index, inplace=True)  # Example: drop the invalid polygon
            # Alternative: You could attempt to repair the geometry using shapely operations.

    # Proceed if you have valid geometries:
    if not gdf.empty:
        geojson = gdf.iloc[0].geometry.__geo_interface__
        return ee.Geometry(geojson)
    else:
        raise ValueError("The shapefile contains no valid polygons.")

In [ ]:
shapefile_folder_path = "/content/drive/MyDrive/Colab Notebooks/Export_Polygon.shp"
roi = shapefile_to_ee_geometry(shapefile_folder_path)

In [ ]:
# Install geemap and import modules
!pip install -U geemap
import ee
import geemap
import geopandas as gpd
from datetime import datetime  # Import the datetime module

# Load the shapefile using geopandas
shapefile_path = "/content/drive/MyDrive/Colab Notebooks/Export_Polygon.shp"
gdf = gpd.read_file(shapefile_path)

# Convert the shapefile to an Earth Engine FeatureCollection
features = []
for i in range(len(gdf)):
    geom = gdf.iloc[i].geometry.__geo_interface__
    feature = ee.Feature(ee.Geometry(geom))
    features.append(feature)

fc = ee.FeatureCollection(features)

# Get the geometry of the first feature (assuming your shapefile has one polygon)
roi = fc.first().geometry()

# Load data and calculate NDVI
# Updated to use Landsat Collection 2, Tier 1, Surface Reflectance
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi).first()
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI') # Updated band names for Collection 2
ndvi_clipped = ndvi.clip(roi)

# Visualize NDVI using geemap
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add Google Earth imagery as a basemap
Map.add_basemap('SATELLITE')

# Define NDVI visualization parameters
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'yellow', 'green', 'green']
}

# Add layers
Map.addLayer(ndvi_clipped, ndvi_vis, 'NDVI')
Map.addLayer(roi, {}, 'ROI')

# Add layer control
Map.addLayerControl()

# Export NDVI as a GeoTIFF
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

export_params = {
    'image': ndvi_clipped,
    'description': f'UVS_NDVI_{current_datetime}',
    'scale': 10,
    'region': roi.getInfo()['coordinates'],
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
    'folder': 'Your_GEE_Exports_Folder'
}

# Start the export task
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Display the map (for Jupyter Notebook or scripts)
Map.to_html('output_map.html')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00


In [ ]:
# Load the shapefile using geopandas
shapefile_path = "/content/drive/MyDrive/Colab Notebooks/Export_Polygon.shp"
gdf = gpd.read_file(shapefile_path)

# Convert the shapefile to an Earth Engine FeatureCollection
features = []
for i in range(len(gdf)):
    geom = gdf.iloc[i].geometry.__geo_interface__
    feature = ee.Feature(ee.Geometry(geom))
    features.append(feature)

fc = ee.FeatureCollection(features)

# Get the geometry of the first feature (assuming your shapefile has one polygon)
roi = fc.first().geometry()

# Load data and calculate NDVI
# Updated to use Landsat Collection 2, Tier 1, Surface Reflectance
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi).first()
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI') # Updated band names for Collection 2
ndvi_clipped = ndvi.clip(roi)

# Visualize NDVI using geemap
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add Google Earth imagery as a basemap
Map.add_basemap('SATELLITE')

# Define NDVI visualization parameters
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'yellow', 'green', 'green']
}

# Add layers
Map.addLayer(ndvi_clipped, ndvi_vis, 'NDVI')
Map.addLayer(roi, {}, 'ROI')

# Add layer control
Map.addLayerControl()

# Export NDVI as a GeoTIFF
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

export_params = {
    'image': ndvi_clipped,
    'description': f'UVS_NDVI_{current_datetime}',
    'scale': 10,
    'region': roi, # Use the ee.Geometry object directly
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
    'folder': 'GEE_Exports' # Specify the folder name in your Google Drive
}

# Start the export task
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Display the map (for Jupyter Notebook or scripts)
Map.to_html('output_map.html')

print(f"Exporting NDVI to Google Drive. Task ID: {task.id}") # Print task ID for monitoring

Exporting NDVI to Google Drive. Task ID: IRXDRWPVNKWZZNHIVBCH4WUR


In [ ]:
# Convert the shapefile to an Earth Engine FeatureCollection
features = []
for i in range(len(gdf)):
    geom = gdf.iloc[i].geometry.__geo_interface__
    feature = ee.Feature(ee.Geometry(geom))
    features.append(feature)

fc = ee.FeatureCollection(features)

# Get the geometry of the first feature (assuming your shapefile has one polygon)
roi = fc.first().geometry()

# Load data and calculate NDVI
# Updated to use Landsat Collection 2, Tier 1, Surface Reflectance
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi).first()
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI') # Updated band names for Collection 2
ndvi_clipped = ndvi.clip(roi)

# Visualize NDVI using geemap
Map = geemap.Map()
Map.centerObject(roi, 10)

# Add Google Earth imagery as a basemap
Map.add_basemap('SATELLITE')

# Define NDVI visualization parameters
ndvi_vis = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'yellow', 'green', 'green']
}

# Add layers
Map.addLayer(ndvi_clipped, ndvi_vis, 'NDVI')
Map.addLayer(roi, {}, 'ROI')

# Add layer control
Map.addLayerControl()

# Export NDVI as a GeoTIFF
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

export_params = {
    'image': ndvi_clipped,
    'description': f'UVS_NDVI_{current_datetime}',
    'scale': 10,
    'region': roi, # Use the ee.Geometry object directly
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
    'folder': 'GEE_Exports' # Specify the folder name in your Google Drive
}

# Start the export task
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# Display the map (for Jupyter Notebook or scripts)
Map.to_html('output_map.html')

print(f"Exporting NDVI to Google Drive. Task ID: {task.id}") # Print task ID for monitoring

Exporting NDVI to Google Drive. Task ID: YPYYUNOI3UEKOTBGQRM7VBYD
